In [1]:
# pip install --upgrade git+https://github.com/hyk2202/helper.git

import pandas as pd
from helper.util import *
from helper.analysis import *
from helper.tensor import *
from helper.plot import *
from helper.timeserise import *
from helper.core import *
from helper.classification import *
from helper.re import *
from helper.regression import *

2024-05-09 17:28:38.940049: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 17:28:38.940071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 17:28:38.940654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 17:28:38.944091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 17:28:39.507950: W tensorflow/compiler/tf2

In [2]:
origin = my_read_data('데이터_전처리_최종.xlsx',info=False, index_col='IDX')

In [3]:
df = origin.copy()
df.columns

Index(['발생날짜', '요일', '공휴일여부', '오전오후', '시간대', '시간', '사고인지 시간', '공공/민간 구분', '날씨',
       '기온', '습도', '인적사고', '보호(방호)조치여부 - 안전방호', '보호(방호)조치여부 - 개인방호', '공종_대분류',
       '공종_중분류', '사고객체_대분류', '사고객체_중분류', '작업프로세스', '장소_대분류', '장소_중분류',
       '부위_대분류', '부위_중분류', '사고원인_주원인', '사상자수', '사고신고사유_부상', '사고신고사유_사망',
       '사고신고사유_물적', '시', '전체공사비', '전체공사비_분류', '해당공종_공사비', '구체적 사고원인',
       '사고발생후 조치사항', '공사기간_시작일', '공사기간_종료일', '공종_시작일', '공종_종료일', '재발방지대책',
       '공정률', '안전관리계획', '설계안전성검토', '작업자수', '지점 번호', '관측소', '기온.1', '강수량', '풍속',
       '풍향', '습도.1', '증기압', '이슬점온도', '현지기압', '해면기압', '일조', '전운량', '중하층운량',
       '시정', '지면온도'],
      dtype='object')

In [4]:
df.drop(['발생날짜','지점 번호','관측소'],axis=1,inplace=True)

In [5]:
temp = []
for i in df['인적사고']:
    if i == '기타':temp.append('분류불능')
    elif i == '없음':temp.append(None)
    else:temp.append(i)

df['인적사고'] = temp
df['인적사고'].value_counts()

넘어짐       5814
떨어짐       4321
물체에 맞음    3509
끼임        2576
분류불능      2152
부딪힘       1850
절단, 베임    1688
깔림         521
질병         381
찔림         307
화상         186
교통사고       123
감전          48
질식          18
Name: 인적사고, dtype: int64

In [6]:
temp =[]
for i in df['강수량']:
    if i>=0.1 : temp.append(1)
    else:temp.append(0)


In [7]:
# 결측치 : 0 정규작업 :1 그외: 2

temp = []
for i in df['사고인지 시간'].fillna(0):
    if i == 0 : temp.append(0)
    elif i == '정규작업 ': temp.append(1)
    else:temp.append(2)

df['사고인지 시간'] = temp
df['사고인지 시간'].value_counts()

1    18032
0     3167
2     2523
Name: 사고인지 시간, dtype: int64

In [8]:
# 결측치 : 0, 조치/해당없음:1, 비조치:2

temp = []
for i in df["보호(방호)조치여부 - 안전방호"].fillna(0):
    if i == 0 :temp.append(0)
    elif i == '비조치': temp.append(2)
    else:temp.append(1)
df["보호(방호)조치여부 - 안전방호"] =temp


df["보호(방호)조치여부 - 안전방호"].value_counts()

1    17153
0     5411
2     1158
Name: 보호(방호)조치여부 - 안전방호, dtype: int64

In [9]:
# 결측치 : 0, 조치/해당없음:1, 비조치:2

temp = []
for i in df["보호(방호)조치여부 - 개인방호"].fillna(0):
    if i == 0 :temp.append(0)
    elif i == '비조치': temp.append(2)
    else:temp.append(1)
df["보호(방호)조치여부 - 개인방호"] =temp


df["보호(방호)조치여부 - 개인방호"].value_counts()

1    17455
0     5410
2      857
Name: 보호(방호)조치여부 - 개인방호, dtype: int64

In [10]:
# 통신설비, 산업설비, 전기설비 => 기타
temp = []
for i in df['공종_대분류']:
    if i in ['통신설비','산업설비','전기설비']:temp.append('기타')
    else:temp.append(i)
df['공종_대분류'] =temp

In [11]:
# 비조치 :2 , 조치,해당없음 : 1 ,결측치 : 0
temp = []
for i in origin["작업프로세스"]:
    if i == '비조치': temp.append(2)
    elif i ==0:temp.append(1)
    else:temp.append(0)

In [12]:
#노동부고시 제2002-8호, 『화학물질 및 물리적인자의 노출기준』 중 <별표 4>의 고온의 노출기준
temp = []
for i in origin['지면온도']:
    if i <25.0: temp.append(0)
    elif i <25.9:temp.append(1)
    elif i<27.9:temp.append(2)
    elif i <30:temp.append(3)
    else:temp.append(4)
df['지면온도'] = temp

In [13]:
df['전체공사비_분류'] = df['전체공사비_분류'].map({'2,000만 ~ 5억원 미만':2, '2,000만 미만':1, '50억원 이상':4, '5억 ~ 50억원 미만':3, '분류불능':0})
df['전체공사비_분류'].value_counts()

4    15759
2     3961
3     3654
1      228
0      120
Name: 전체공사비_분류, dtype: int64

In [14]:
temp = []
for i in df['작업자수']:
    try:
        a = int(i[:3])
        if a == 100: temp.append(4)
        elif a == 300: temp.append(5)
        elif a == 500: temp.append(6)

    except:
        a = int(i[:2])
        if a ==19:temp.append(0)
        elif a == 20 :temp.append(1)
        elif a==50:temp.append(2)
df['작업자수'] = temp


In [15]:
temp = []
for i in origin['공정률']:
    try:
        if '미만' in i : temp.append(0)
        elif '이상' in i: temp.append(9)
        else:
            temp.append(int(i[:1]))
    except:
        
        temp.append(0)

df['공정률'] = temp

In [16]:
df.dtypes

    
fields=[]
for f in df.columns:
    if df[f].dtypes not in [
        "int",
        "int32",
        "int64",
        "float",
        "float32",
        "float64",
    ]:
        fields.append(f)

In [17]:
cate = df[fields]
df2 = df.drop(fields,axis=1)

In [18]:
df2 = my_replace_missing_value(df2,'constant',0)
cate = my_replace_missing_value(cate,'constant','0')

In [19]:
df2.columns

Index(['시간대', '사고인지 시간', '습도', '보호(방호)조치여부 - 안전방호', '보호(방호)조치여부 - 개인방호',
       '사상자수', '전체공사비_분류', '공정률', '작업자수', '기온.1', '강수량', '풍속', '풍향', '습도.1',
       '증기압', '이슬점온도', '현지기압', '해면기압', '일조', '전운량', '중하층운량', '시정', '지면온도'],
      dtype='object')

In [20]:
fields.remove('공사기간_시작일')
fields.remove('공사기간_종료일')
fields.remove('공종_시작일')
fields.remove('공종_종료일')
fields.remove('구체적 사고원인')
fields.remove('사고발생후 조치사항')
fields.remove('재발방지대책')

In [21]:
drop_fields = ['시간대','습도','']

In [22]:
df2.columns

Index(['시간대', '사고인지 시간', '습도', '보호(방호)조치여부 - 안전방호', '보호(방호)조치여부 - 개인방호',
       '사상자수', '전체공사비_분류', '공정률', '작업자수', '기온.1', '강수량', '풍속', '풍향', '습도.1',
       '증기압', '이슬점온도', '현지기압', '해면기압', '일조', '전운량', '중하층운량', '시정', '지면온도'],
      dtype='object')

In [23]:
cate = cate[fields]

In [24]:
df2[fields]=cate
df2.dtypes
fields.remove('인적사고')


In [25]:
df2 = df2.dropna()

In [26]:


x = my_dummies(df2,*fields)
y = my_labelling(df2[['인적사고']],'인적사고')


In [27]:
x.shape

(23526, 1746)

In [28]:
x['인적사고'] = y

In [29]:
x_train,x_test,y_train,y_test = my_train_test_split(x,'인적사고')
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((18820, 1745), (4706, 1745), (18820,), (4706,))

In [30]:
params = {
    "n_estimators": [10, 20, 50, 100],
    "criterion": ["squared_error", "absolute_error", "friedman_mse", "poisson"],
    "max_features": ["sqrt", "log2"],
    "max_depth": [10, 20, 50, None],
}

xgb - y 갯수 줄이기 전

{'subsample': 1, 'reg_lambda': 7, 'reg_alpha': 1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 2, 'colsample_bytree': 0.7}

[분류분석 성능평가]


| | 훈련데이터 |   검증데이터 | 설명                                                      |
|--|--|--|--|
| 정확도(Accuracy)  |     0.541919 |     0.488936 | 예측 결과(TN,FP,TP,TN)가 실제 결과(TP,TN)와 일치하는 정도 |
| 정밀도(Precision) |     0.655207 |     0.420337 | 양성으로 예측한 결과(TP,FP) 중 실제 양성(TP)인 비율       |
| 재현율(Recall)    |     0.541919 |     0.335646 | 실제 양성(TP,FN) 중 양성(TP)으로 예측한 비율              |
| F1 Score          |     0.409932 |     0.357504 | 정밀도와 재현율의 조화평균                                |
| AUC(ovo)          |     0.917224 |     0.868432 | One vs One에 대한 AUC로, 1에 가까울수록 좋은 모델         |
| AUC(ovr)          |     0.913331 |     0.860516 | One vs Rest에 대한 AUC로, 1에 가까울수록 좋은 모델        |



[Best Params]
{'subsample': 1, 'reg_lambda': 7, 'reg_alpha': 1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 2, 'colsample_bytree': 0.7}



|                   |   훈련데이터 |   검증데이터 | 설명                                                      |
|--|--|--|--|
| 정확도(Accuracy)  |     0.545696 |     0.509137 | 예측 결과(TN,FP,TP,TN)가 실제 결과(TP,TN)와 일치하는 정도 |
| 정밀도(Precision) |     0.675546 |     0.469469 | 양성으로 예측한 결과(TP,FP) 중 실제 양성(TP)인 비율       |
| 재현율(Recall)    |     0.545696 |     0.361035 | 실제 양성(TP,FN) 중 양성(TP)으로 예측한 비율              |
| F1 Score          |     0.416669 |     0.381702 | 정밀도와 재현율의 조화평균                                |
| AUC(ovo)          |     0.894882 |     0.868483 | One vs One에 대한 AUC로, 1에 가까울수록 좋은 모델         |
| AUC(ovr)          |     0.895783 |     0.861934 | One vs Rest에 대한 AUC로, 1에 가까울수록 좋은 모델        |

In [31]:
param_xgb = {'subsample': 1, 'reg_lambda': 7, 'reg_alpha': 1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 2, 'colsample_bytree': 0.7}

In [32]:
xgb = XGBClassifier(**param_xgb)

In [33]:
xgb.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=2, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [34]:
accuracy_score(y_test, xgb.predict(x_test))

0.5125371865703358

In [35]:
import sys
if sys.platform == 'linux':
  sys.path.append("/home/yjg1005/nas/Python/Mega_IT")
else:
  sys.path.append("Y:/Python/Mega_IT")

from yjk_module.preprocessing import *
from yjk_module.ml_module import *
from yjk_module.dl_module import *

In [63]:
param_lgbm = {'num_leaves' : [31, 20], 'learning_rate' : [0.01, 0.05], 'max_depth' : [5, 7, 9], 'subsample' : [0.8, 0.9], 'colsample_bytree' : [0.6, 0.8]}

In [37]:
from sklearn.model_selection import ParameterGrid

In [65]:
params_lgbm = ParameterGrid(param_lgbm)

In [66]:
# df_lgbm = pd.DataFrame()

# for i, v in enumerate(params_lgbm):
#     print(v)
#     lgbm = LGBMClassifier(**v, verbose = -1)
#     lgbm.fit(x_train.to_numpy(), y_train)
#     df_lgbm.loc[i, 'score'] = lgbm.score(x_test.to_numpy(), y_test)
#     df_lgbm.loc[i, 'model'] = lgbm
#     df_lgbm.loc[i, 'param'] = str(v)

{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 5, 'num_leaves': 31, 'subsample': 0.8}
{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 5, 'num_leaves': 31, 'subsample': 0.9}
{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 5, 'num_leaves': 20, 'subsample': 0.8}
{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 5, 'num_leaves': 20, 'subsample': 0.9}
{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 7, 'num_leaves': 31, 'subsample': 0.8}
{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 7, 'num_leaves': 31, 'subsample': 0.9}
{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 7, 'num_leaves': 20, 'subsample': 0.8}
{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 7, 'num_leaves': 20, 'subsample': 0.9}
{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 9, 'num_leaves': 31, 'subsample': 0.8}
{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 9, 'num_leaves': 31, 'subsamp

In [70]:
# df_lgbm.sort_values('score', ascending = False).iloc[0, 2]
lgbm_param = {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 7, 'num_leaves': 20, 'subsample': 0.9}

In [73]:
lgbm = LGBMClassifier(**lgbm_param, verbose = -1)

In [74]:
lgbm.fit(x_train.to_numpy(), y_train)

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.05, max_depth=7,
               num_leaves=20, subsample=0.9, verbose=-1)

In [75]:
lgbm.score(x_test.to_numpy(), y_test)

0.509987250318742

In [78]:
param_catb = {'depth' : [3, 5, 7, 9], 'learning_rate' : [0.001, 0.009, 0.1], 'random_strength' : [4, 6, 8]}

In [79]:
params_catb = ParameterGrid(param_catb)

In [81]:
df_catb = pd.DataFrame()

for i, v in enumerate(params_catb):
    print(v)
    catb = CatBoostClassifier(**v, verbose = 0)
    catb.fit(x_train.to_numpy(), y_train)
    df_catb.loc[i, 'score'] = catb.score(x_test.to_numpy(), y_test)
    df_catb.loc[i, 'model'] = catb
    df_catb.loc[i, 'param'] = str(v)

{'depth': 3, 'learning_rate': 0.001, 'random_strength': 4}
{'depth': 3, 'learning_rate': 0.001, 'random_strength': 6}
{'depth': 3, 'learning_rate': 0.001, 'random_strength': 8}
{'depth': 3, 'learning_rate': 0.009, 'random_strength': 4}
{'depth': 3, 'learning_rate': 0.009, 'random_strength': 6}
{'depth': 3, 'learning_rate': 0.009, 'random_strength': 8}
{'depth': 3, 'learning_rate': 0.1, 'random_strength': 4}
{'depth': 3, 'learning_rate': 0.1, 'random_strength': 6}
{'depth': 3, 'learning_rate': 0.1, 'random_strength': 8}
{'depth': 5, 'learning_rate': 0.001, 'random_strength': 4}
{'depth': 5, 'learning_rate': 0.001, 'random_strength': 6}
{'depth': 5, 'learning_rate': 0.001, 'random_strength': 8}
{'depth': 5, 'learning_rate': 0.009, 'random_strength': 4}
{'depth': 5, 'learning_rate': 0.009, 'random_strength': 6}
{'depth': 5, 'learning_rate': 0.009, 'random_strength': 8}
{'depth': 5, 'learning_rate': 0.1, 'random_strength': 4}
{'depth': 5, 'learning_rate': 0.1, 'random_strength': 6}
{'depth

In [88]:
# df_catb.sort_values('score', ascending = False).iloc[0, 2]
catb_param = {'depth': 5, 'learning_rate': 0.1, 'random_strength': 8}

In [89]:
catb = CatBoostClassifier(**catb_param, verbose = 0)

In [90]:
catb.fit(x_train, y_train)

In [91]:
catb.score(x_test, y_test)

0.5106247343816405

In [103]:
# param_rf = {'max_depth' : [None, 5, 7, 10], 'min_samples_split' : [2, 5, 10], 'min_samples_leaf' : [1, 2, 4]}
# params_rf = ParameterGrid(param_rf)

In [ ]:
# df_rf = pd.DataFrame()

# for i, v in enumerate(params_rf):
#     print(v)
#     rf = RandomForestClassifier(**v)
#     rf.fit(x_train.to_numpy(), y_train)
#     df_rf.loc[i, 'score'] = rf.score(x_test.to_numpy(), y_test)
#     # df_rf.loc[i, 'model'] = rf
#     df_rf.loc[i, 'param'] = str(v)

In [109]:
from sklearn.ensemble import VotingClassifier, StackingClassifier

In [113]:
models = [('xgb', xgb), ('lgbm', lgbm), ('catb', catb)]

In [114]:
vote_soft = VotingClassifier(models, voting = 'soft')

In [115]:
vote_soft.fit(x_train.to_numpy(), y_train)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.7, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=2,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None, min_child_weight=1,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='multi:softprob', ...)),
                             ('lgbm', RandomForestClassifier()),
                             ('catb',
                              <catboost.core.CatBoostClassifier object at 0x7fe476caaa70>)],
                 voting='soft')

In [116]:
vote_soft.score(x_test.to_numpy(), y_test)

0.5150871228219295

In [117]:
vote_hard = VotingClassifier(models, voting = 'hard')

In [121]:
stacking = StackingClassifier(models, final_estimator = XGBClassifier())

In [122]:
stacking.fit(x_train.to_numpy(), y_train)

StackingClassifier(estimators=[('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.7, device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=2,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate...
                                                 grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=None, ...))

In [123]:
stacking.score(x_test.to_numpy(), y_test)

0.4887377815554611